# Demographics Data Prep
<font size=4 color='blue'>Project: Congressional Activity</font>
***

**Project Summary:**  
Perform a trend analysis to see if the US Congress has become less productive over time

**Notebook Scope:**  
This notebook includes code to load and tidy raw demographic data from an Excel spreadsheet. Source data can be downloaded from the <a href="https://www.brookings.edu/articles/vital-statistics-on-congress/">Brookings Institute</a>.  

**Output:**  
An Excel file containing scrubbed Demographic data is generated.  
***

***
# Notebook Setup
***

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re

In [2]:
# Set options
pd.options.display.multi_sparse = False
pd.options.display.max_colwidth = 25

In [3]:
%%html
<!-- Prevent text wrappping in dataframe displays for a cleaner print -->
<style> .dataframe td {white-space: nowrap;}</style>

***  
# Read and Tidy Party Data
***

In [4]:
# Read party data for the Senate
file_name = '../data/Vital Statistics on Congress.xlsx'
senate_party_df = pd.read_excel(file_name, sheet_name='1-20', skiprows=4, header=None, usecols='A,C:G', skipfooter=3)
senate_party_df.columns = ['Congress', 'Members', 'Democrats', 'Republicans', 'Other Parties', 'Vacant']
senate_party_df.insert(loc=2, column='Chamber', value='Senate')
senate_party_df.head()

,Congress,Members,Chamber,Democrats,Republicans,Other Parties,Vacant
0,34th,62,Senate,42,15,5.0,NaN
1,35th,64,Senate,39,20,5.0,NaN
2,36th,66,Senate,38,26,2.0,NaN
3,37th,50,Senate,11,31,7.0,1.0
4,38th,51,Senate,12,39,NaN,NaN


In [5]:
# Read party data for the House
file_name = '../Data/Vital Statistics on Congress.xlsx'
house_party_df = pd.read_excel(file_name, sheet_name='1-20', skiprows=4, header=None, usecols='A,I:M', skipfooter=3)
house_party_df.columns = ['Congress', 'Members', 'Democrats', 'Republicans', 'Other Parties', 'Vacant']
house_party_df.insert(loc=2, column='Chamber', value='House')
house_party_df.head()

,Congress,Members,Chamber,Democrats,Republicans,Other Parties,Vacant
0,34th,234,House,83,108,43.0,NaN
1,35th,237,House,131,92,14.0,NaN
2,36th,237,House,101,113,23.0,NaN
3,37th,178,House,42,106,28.0,2.0
4,38th,183,House,80,103,NaN,NaN


In [6]:
# Consolidate Senate and House party data
party_df = pd.concat([senate_party_df, house_party_df]).reset_index(drop=True)

In [7]:
# Infer datatypes and review
party_df = party_df.convert_dtypes()
party_df.dtypes

Congress         string[python]
Members                  object
Chamber          string[python]
Democrats                object
Republicans               Int64
Other Parties             Int64
Vacant                    Int64
dtype: object

In [8]:
# All columns, except Chamber, should be int
# For string columns, remove any non-numeric characters and convert to int
party_df['Congress'] = party_df['Congress'].str.extract(r'(\d*)').astype(int)

# For object columns, remove any non-numeric characters and convert to int
obj_cols = party_df.select_dtypes(include=['object']).columns
for col in obj_cols:
    party_df[col] = party_df[col].astype('str').str.extract(r'(\d*)').astype(int)

# For Int64 columns, replace NA with zero and convert to int
int64_cols = party_df.select_dtypes(include=['Int64']).columns
party_df[int64_cols] = party_df[int64_cols].fillna(0).astype(int)

In [9]:
# Reorder and sort dataframe
party_df = party_df[['Congress', 'Chamber', 'Members', 'Vacant', 'Democrats', 'Republicans', 'Other Parties']].copy()
party_df.sort_values(by=['Congress', 'Chamber'], inplace=True)
party_df.set_index(['Congress', 'Chamber'], drop=True, inplace=True)
party_df.head()

,,Members,Vacant,Democrats,Republicans,Other Parties
Congress,Chamber,,,,,
34,House,234,0,83,108,43
34,Senate,62,0,42,15,5
35,House,237,0,131,92,14
35,Senate,64,0,39,20,5
36,House,237,0,101,113,23


***  
# Read and Tidy Seniority Data
***

In [10]:
# Read seniority data for the House
file_name = '../Data/Vital Statistics on Congress.xlsx'
house_seniority_df = pd.read_excel(file_name, sheet_name='1-6', skiprows=3, usecols='A:D,F:H', skipfooter=3)
house_seniority_df.head()

,Congress,1 term,2 terms,3 terms,4 - 6 terms,7 - 9 terms,10 + terms
0,83rd (1953),NaN,NaN,NaN,NaN,NaN,NaN
1,Percent,18.706697,16.859122,14.7806,27.020785,13.394919,9.237875
2,Seats,81.000000,73.000000,64.0000,117.000000,58.000000,40.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84th (1955),NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# This dataset uses four rows per observation. We'll keep only the 3rd row (raw counts), but we need to carry down the Congress data from the
# first row of each observation
for row in range(len(house_seniority_df)):
    if row == 0 or row % 4 == 0:
        congress = house_seniority_df.at[row, 'Congress']
    elif row % 4 == 2:
        house_seniority_df.at[row, 'Congress'] = congress
rows_to_drop = [x for x in range(len(house_seniority_df)) if x % 4 != 2]
house_seniority_df.drop(index=rows_to_drop, inplace=True)
house_seniority_df.reset_index(drop=True, inplace=True)
house_seniority_df.head()

,Congress,1 term,2 terms,3 terms,4 - 6 terms,7 - 9 terms,10 + terms
0,83rd (1953),81.0,73.0,64.0,117.0,58.0,40.0
1,84th (1955),57.0,73.0,63.0,119.0,73.0,50.0
2,85th (1957),46.0,50.0,66.0,142.0,66.0,63.0
3,86th (1959),82.0,45.0,49.0,136.0,64.0,57.0
4,87th (1961),62.0,65.0,36.0,131.0,76.0,67.0


In [12]:
# Add Chamber column and set to House for all rows
house_seniority_df.insert(loc=1, column='Chamber', value='House')

In [13]:
# Clean up Congress column
house_seniority_df['Congress'] = house_seniority_df['Congress'].replace('[a-z]+.*', '', regex=True)

In [14]:
# Read seniority data for the Senate
file_name = '../Data/Vital Statistics on Congress.xlsx'
senate_seniority_df = pd.read_excel(file_name, sheet_name='1-7', skiprows=3, usecols='A,C:F', skipfooter=3)
senate_seniority_df.head()

,Congress,6 years or less,7 - 12 years,13 - 18 years,19 years or more
0,83rd,46 (16),29,14,7
1,84th,42 (14),37,8,9
2,85th,37 (10),36,13,10
3,86th,42 (20),30,14,12
4,87th,42 (7),25,22,11


In [15]:
# Add Chamber column and set to Senate for all rows
senate_seniority_df.insert(loc=1, column='Chamber', value='Senate')

In [16]:
# Clean up Congress column
senate_seniority_df['Congress'] = senate_seniority_df['Congress'].replace('[a-z]+', '', regex=True)

In [17]:
# For our purposes, we will remove the parenthetical number under the column "6 years or less". This value indicates freshmen senators
senate_seniority_df['6 years or less'] = senate_seniority_df['6 years or less'].str.extract(r'(\d*)').astype(int)

***
**Note:**  
For data consistency, seniority will be measured by years and not terms.
***

In [18]:
# Combine 1 term, 2 terms, and 3 terms columns to align with the Senate dataset
house_seniority_df['6 years or less'] = house_seniority_df[['1 term', '2 terms', '3 terms']].sum(axis=1)
house_seniority_df.drop(['1 term', '2 terms', '3 terms'], axis=1, inplace=True)

In [19]:
# Rename the remaining House columns to match the Senate dataset and
house_seniority_df.rename(columns={'4 - 6 terms': '7 - 12 years', '7 - 9 terms': '13 - 18 years', '10 + terms': '19 years or more'}, inplace=True)

In [20]:
# Consolidate Senate and House seniority data
seniority_df = pd.concat([senate_seniority_df, house_seniority_df]).reset_index(drop=True)

In [21]:
# Infer datatypes and review
seniority_df = seniority_df.convert_dtypes()
seniority_df.dtypes

Congress            string[python]
Chamber             string[python]
6 years or less              Int64
7 - 12 years                 Int64
13 - 18 years                Int64
19 years or more             Int64
dtype: object

In [22]:
# All columns, except Chamber, should be int
# For string columns, remove any non-numeric characters and convert to int
seniority_df['Congress'] = seniority_df['Congress'].str.extract(r'(\d*)').astype(int)

# For Int64 columns, replace NA with zero and convert to int
int64_cols = seniority_df.select_dtypes(include=['Int64']).columns
seniority_df[int64_cols] = seniority_df[int64_cols].fillna(0).astype(int)

In [23]:
# Set index to Congress and Chamber
seniority_df.set_index(['Congress', 'Chamber'], inplace=True)

In [24]:
# Preview seniority dataset
seniority_df.head()

,,6 years or less,7 - 12 years,13 - 18 years,19 years or more
Congress,Chamber,,,,
83,Senate,46,29,14,7
84,Senate,42,37,8,9
85,Senate,37,36,13,10
86,Senate,42,30,14,12
87,Senate,42,25,22,11


***
# Consolidate Data
***

In [27]:
# Since the party dataframe has the largest range of data in terms of Congresses, that will be our primary dataframe for joins
demographics_df = party_df.join([seniority_df])

In [28]:
# Fill na and convert to int
demographics_df = demographics_df.fillna(0).astype('int')
demographics_df.head()

,,Members,Vacant,Democrats,Republicans,Other Parties,6 years or less,7 - 12 years,13 - 18 years,19 years or more
Congress,Chamber,,,,,,,,,
34,House,234,0,83,108,43,0,0,0,0
34,Senate,62,0,42,15,5,0,0,0,0
35,House,237,0,131,92,14,0,0,0,0
35,Senate,64,0,39,20,5,0,0,0,0
36,House,237,0,101,113,23,0,0,0,0


***
# Write to Excel
***

In [29]:
demographics_df.reset_index(inplace=True)
demographics_df.to_excel('../data/demographics_data.xlsx', index=False)

***
**End**
***